# 1. Configuration

### Imports

In [1]:
import pygad
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

### Read the data

In [2]:
dataset = pd.read_csv("dataset.csv")
dataset

,Equation,Xs,Ys
0,((x ** 4) - 6),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-5, 10, 75, 250, 619, 1290, 2395, 4090, 6555,..."
1,(((x / 8) * 2) + 1),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3..."
2,(((x - 1) - 3) / 5),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1...."
3,(x * 5),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60..."
4,(x + 2),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
...,...,...,...
93,1*x**5 + -2*x**3 + -1*x + -5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-7, 9, 181, 887, 2865, 7333, 16109, 31731, 57..."
94,3*x**5 + 2*x**4 + 1*x + -2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4, 128, 892, 3586, 10628, 25924, 55228, 10650..."
95,-4*x**5 + -1*x + -5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-10, -135, -980, -4105, -12510, -31115, -6724..."
96,4*x + +1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49,..."


# 2. Representation

### Tree data structure

#### Node class

In [3]:
class Node:
    def __init__(self, value=None):
        self.value = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)

#### Equation tree class

The `EquationTree` class is a custom implementation of a binary tree data structure that represents a mathematical equation. It is used to parse and evaluate mathematical expressions in a tree-like structure. 

The class has the following methods:

- `build_tree(self, expression)`: Builds the tree structure from a given mathematical expression.
- `print_tree(self)`: Prints the tree structure in a readable format.
- `evaluate(self, x)`: Evaluates the mathematical expression represented by the tree structure for a given value of `x`.


In [76]:
class EquationTree:
    def __init__(self, root=None):
        self.root = root

    def build_tree(self, expression):
        self.root = self._build_tree_recursive(expression)

    def _build_tree_recursive(self, expression):
        
        current_depth = -1  #odvisn v kok ( gres notr

        lowest_depth = 0    #najvecja globina
        lowest_depth_index = -1 #index najvecje globine, oz tm k je element d splitamo drevo

        leaf = False    #ce je list pa v izrazu ni vec operatorjev pol vrne list
        leaf_value = None   #vrednost lista

        priority_lvl = 0   #to je zto d ma ** prednost pred * in /, pa + in - pred tem

        for i in range(len(expression)):
            if expression[i] == '(':    
                current_depth += 1  #povecamo globino
            elif expression[i] == ')':
                current_depth -= 1  #zmanjsamo globino
            elif expression[i] in ['+', '-']:
                #   ce smo v najmajsi globini    ce sploh ni globine v enacbi    prednost pred drugimi operatorji (kle nima tok veze, bl je spodi)
                if current_depth < lowest_depth or lowest_depth_index == -1 or priority_lvl > 1:  
                    lowest_depth = current_depth
                    lowest_depth_index = i
                    priority_lvl = 1 
            elif expression[i] in ['*', '/']:
                if current_depth < lowest_depth or lowest_depth_index == -1 or priority_lvl > 2:
                    lowest_depth = current_depth
                    lowest_depth_index = i
                    priority_lvl = 2
            elif expression[i] in ['**']:
                if current_depth < lowest_depth or lowest_depth_index == -1 or priority_lvl > 3:
                    lowest_depth = current_depth
                    lowest_depth_index = i
                    priority_lvl = 3
            elif expression[i] == 'x' or expression[i].lstrip('-').isdigit():
                leaf_value = expression[i]
                leaf = True

        if lowest_depth_index == -1 and leaf:
            return Node(leaf_value)
        else:
            root = Node(expression[lowest_depth_index])
            # print(expression[:lowest_depth_index])
            # print(expression[lowest_depth_index+1:])
            root.add_child(self._build_tree_recursive(expression[:lowest_depth_index])) #rekurzivno zgradimo levo poddrevo
            root.add_child(self._build_tree_recursive(expression[lowest_depth_index+1:]))   #rekurzivno zgradimo desno poddrevo

            return root
        
    def print_tree(self):
        self._print_tree_recursive(self.root)
    
    def _print_tree_recursive(self, root, depth=0):
        if root is None:
            return

        print('\t' * depth + str(root.value) + "--------")

        for child in root.children:
            self._print_tree_recursive(child, depth + 1)

    def evaluate(self, x):
        return self._evaluate_recursive(self.root, x)
    
    def _evaluate_recursive(self, root, x):
        if root is None:
            return None

        if root.value == 'x':
            return x
        elif root.value.lstrip('-').isdigit():
            return int(root.value)
        elif root.value == '+':
            return self._evaluate_recursive(root.children[0], x) + self._evaluate_recursive(root.children[1], x)
        elif root.value == '-':
            return self._evaluate_recursive(root.children[0], x) - self._evaluate_recursive(root.children[1], x)
        elif root.value == '*':
            return self._evaluate_recursive(root.children[0], x) * self._evaluate_recursive(root.children[1], x)
        elif root.value == '/':
            return self._evaluate_recursive(root.children[0], x) / self._evaluate_recursive(root.children[1], x)
        elif root.value == '**':
            return self._evaluate_recursive(root.children[0], x) ** self._evaluate_recursive(root.children[1], x)
        else:
            return None

Helper function to convert a string equation into an array of `tokens`.

In [79]:
def stringEQtoArray(equation):
    arr = np.array([])
    skip = 0

    for index, char in enumerate(equation):
        if skip != 0:
            skip-=1
            continue
    
        if char == ' ':
            continue
        elif char == 'x' or char.isdigit():
            arr = np.append(arr, char)
        elif char == '*' and equation[index+1] == '*':
            arr = np.append(arr, '**')
            skip = 1
        elif char in ['+', '-'] and equation[index+1].isdigit():
            if char == '+':
                arr = np.append(arr, equation[index+1])
            else:
                arr = np.append(arr, char+equation[index+1])
            skip = 1
        elif char in ['+', '-', '*', '/', '(', ')']:
            arr = np.append(arr, char)

    return arr

Helper function to convert a string sequance array to an array.

In [58]:
def stringArrayToArray(string):
    string = string[1:-1]
    arr = string.split(',')
    arr = [float(i) for i in arr]
    return arr

In [78]:
#vse dela prou :)

for x in range(0,1):
    equation_inputs = dataset.iloc[x].values[0]
    #print(equation_inputs)
    equation = stringEQtoArray(equation_inputs)
    #print(equation)
    equation_tree = EquationTree()
    equation_tree.build_tree(equation) 
    equation_tree.print_tree()

    outputs = stringArrayToArray(dataset.iloc[x].values[2])
    sum = 0
    for i in range(100):
        sum += (equation_tree.evaluate(i+1) - outputs[i])

    #print(sum) #more bit 0
    

((x ** 4) - 6)
['(' '(' 'x' '**' '4' ')' '-' '6' ')']
---------
	**--------
		x--------
		4--------
	6--------


### 3. Genetic algorithm

In [80]:
#trying for the first equation first
true_equation = stringEQtoArray(dataset.iloc[0].values[0])
inputs = stringArrayToArray(dataset.iloc[0].values[1])
outputs = stringArrayToArray(dataset.iloc[0].values[2])

print(true_equation)
print(inputs)
print(outputs)

['(' '(' 'x' '**' '4' ')' '-' '6' ')']
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0]
[-5.0, 10.0, 75.0, 250.0, 619.0, 1290.0, 2395.0, 4090.0, 6555.0, 9994.0, 14635.0, 20730.0, 28555.0, 38410.0, 50619.0, 65530.0, 83515.0, 104970.0, 130315.0, 159994.0, 194475.0, 234250.0, 279835.0, 331770.0, 390619.0, 456970.0, 531435.0, 614650.0, 707275.0, 809994.0, 923515.0, 1048570.0, 1185915.0, 1336330.0, 1500619.0, 1679610.0, 1874155.0, 2085130.0, 2313435.0, 2559

In [ ]:
def model(equation):
    equation_tree = EquationTree()
    equation_tree.build_tree(equation)

    return np.array([equation_tree.evaluate(i+1) for i in range(100)])


In [81]:
def fitness_func(ga_instance, solution, solution_idx):
    
    model_outputs = model(solution)

    #square the difference between the outputs and the actual values
    return -np.sum(np.square(model_outputs - outputs))

In [84]:
#try the crossover function to only add the parents
def crossover_func(parents, offspring_size, ga_instance):
    
    child = np.array(parents[0],['+'],parents[1])

    return child

In [85]:
#try the mutation function to only change the operators
def mutation_func(offspring, ga_instance):
    
    for i in range(len(offspring)):
        if offspring[i] in ['+', '-', '*', '/']:
            offspring[i] = np.random.choice(['+', '-', '*', '/'])

    return offspring

In [86]:
initial_population = [['x'],np.linspace(1,100,100)]

In [87]:
#try the ga
ga_instance = pygad.GA(num_generations=100,
                        num_parents_mating=2,
                        fitness_func=fitness_func,
                        sol_per_pop=10,
                        num_genes=100,
                        gene_type=np.ndarray,
                        gene_space=['x', '+', '-', '*', '/'],
                        crossover_type=crossover_func,
                        mutation_type=mutation_func,
                        initial_population=initial_population)

ga_instance.run()

Unexpected type <class 'str'> for the element indexed 0 of 'gene_space'. The accepted types are list/tuple/range/numpy.ndarray of numbers, a single number (int/float), or None.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygad/pygad.py", line 244, in __init__
    raise TypeError(f"Unexpected type {type(el)} for the element indexed {index} of 'gene_space'. The accepted types are list/tuple/range/numpy.ndarray of numbers, a single number (int/float), or None.")
TypeError: Unexpected type <class 'str'> for the element indexed 0 of 'gene_space'. The accepted types are list/tuple/range/numpy.ndarray of numbers, a single number (int/float), or None.


AttributeError: 'tuple' object has no attribute 'tb_frame'